# 前処理大全 SQL+Python版 (Chapter9)

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
%load_ext sql
import os

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# MagicコマンドでSQLを書くための設定
%sql $dsl

'Connected: padawan@dsdojo_db'

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
customer_df = pd.read_csv("./data/customer.csv")
reserve_df = pd.read_csv("./data/reserve.csv")
production_df = pd.read_csv("./data/production.csv")
production_missin_df = pd.read_csv("./data/production_missing_num.csv")
production_missin_category_df = pd.read_csv("./data/production_missing_category.csv")

# 演習問題

## 冒頭

- カテゴリ型とは取りうる値が決まっている値（？）です。
- 例
  - 都道府県
  - 会員である・会員でない（フラグ等）
    - 2値の場合はbool型とも呼ぶ
- 連続値などもカテゴリに変換できる
  - 年齢の数値⇒20代、30代、...に変換
- カテゴリ型はカテゴリ間で演算できない
  - 数値をカテゴリに変換する場合、関係性が消える。
    - 大小関係を明示的に表現できなくなる。
    - 20代のカテゴリと30代のカテゴリの引き算はできない。

### 009_category/001: カテゴリ型への変換

- 元データは文字列型、数値型であることが多いので、カテゴリ型へ変換する。
- カテゴリ型は、値とインデックスで構成される。

<span style="background-color:yellow;">問１：custemerテーブルの性別をブール型とカテゴリ型に変換せよ</span>

#### SQL (Awesome)

- CASEを使う。

In [3]:
%%sql
SELECT
    CASE WHEN sex = 'man' THEN
        TRUE
    ELSE
        FALSE
    END AS sex_is_man
FROM
    customer_tb
LIMIT 10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sex_is_man
True
True
False
True
True
True
True
False
False
False


- カテゴリ型の場合
  - CASEよりこっちの方が汎用性が高いのかな？

In [4]:
%%sql
with
sex_mst as (
    select
        sex
        , row_number() OVER() as sex_mst_id
    from
        customer_tb
    group by
        sex
)
select
    base.*
    , sex_mst.sex_mst_id
from
    customer_tb as base
inner join
    sex_mst
    on
        base.sex = sex_mst.sex
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,age,sex,home_latitude,home_longitude,sex_mst_id
c_1,41,man,35.092193,136.512347,2
c_2,38,man,35.325076,139.410551,2
c_3,49,woman,35.120543,136.511179,1
c_4,43,man,43.034868,141.240314,2
c_5,31,man,35.102661,136.523797,2
c_6,52,man,34.440768,135.390487,2
c_7,50,man,43.015758,141.231321,2
c_8,65,woman,38.201268,140.465961,1
c_9,36,woman,33.3228,130.330689,1
c_10,34,woman,34.290414,132.302601,1


#### R (Awesome)

- as.logical関数でlogical型、factor関数でfactor型に変換する。
- factor型は、as.numeric関数でインデックスを取得でき、levels関数で値（マスターデータ）にアクセスできる。

#### Python (Awesome)

- bool型とcategory型が提供されており、astypeで変換できる。

In [5]:
customer_df['sex_is_man'] = customer_df['sex']=='man' # 本にもあるように、astypeなくてもOK
customer_df.head(10)

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man
0,c_1,41,man,35.092193,136.512347,True
1,c_2,38,man,35.325076,139.410551,True
2,c_3,49,woman,35.120543,136.511179,False
3,c_4,43,man,43.034868,141.240314,True
4,c_5,31,man,35.102661,136.523797,True
5,c_6,52,man,34.440768,135.390487,True
6,c_7,50,man,43.015758,141.231321,True
7,c_8,65,woman,38.201268,140.465961,False
8,c_9,36,woman,33.322800,130.330689,False
9,c_10,34,woman,34.290414,132.302601,False


- category型の場合

In [6]:
customer_df['sex_c'] = pd.Categorical(customer_df['sex'], categories=['man','woman'])
customer_df.head(10)

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c
0,c_1,41,man,35.092193,136.512347,True,man
1,c_2,38,man,35.325076,139.410551,True,man
2,c_3,49,woman,35.120543,136.511179,False,woman
3,c_4,43,man,43.034868,141.240314,True,man
4,c_5,31,man,35.102661,136.523797,True,man
5,c_6,52,man,34.440768,135.390487,True,man
6,c_7,50,man,43.015758,141.231321,True,man
7,c_8,65,woman,38.201268,140.465961,False,woman
8,c_9,36,woman,33.322800,130.330689,False,woman
9,c_10,34,woman,34.290414,132.302601,False,woman


In [7]:
print("="*40)
print(customer_df['sex_c'].cat.codes)
print("="*40)
print(customer_df['sex_c'].cat.categories)

0      0
1      0
2      1
3      0
4      0
      ..
995    0
996    0
997    1
998    1
999    0
Length: 1000, dtype: int8
Index(['man', 'woman'], dtype='object')


### 009_category/002:ダミー変数化

- category型は便利だが、対応していないライブラリもある。(実際、私も使ったことがない…)
- その場合、カテゴリをフラグの集合値に変換する。これをダミー変数化という。

|||||
|:---|:---|:---|:---|
|ホテルA|1|0|0|
|ホテルB|0|1|0|
|ホテルC|0|0|1|

- OneHotベクトルと同じことを指している。
- ダミー変数は、１つのカラムに対して、その種類数まで増える。
- 実際に機械学習する場合は、種類数-1とすることもできるが、ダミー変数の重要度を調べにくくなる。
  - 何が言いたいかというと、そこまで変わらないので、種類数-1にわざわざしなくてもいいってことかな。

|||||
|:---|:---|:---|:---|
|ホテルA|0|0|
|ホテルB|1|0|
|ホテルC|0|1|

- どうでもいいけど、ダミーフラグがダミープラグって似てるよね。

<span style="background-color:yellow;">問２：性別をダミー変数に変換せよ</span>

#### SQL (Not Awesome)

- カテゴリが増えるたびにcase文が増える。
- Not Awesomeになっているけど、SQLでやらざるを得ない致し方ないケースもあるので、使い方としては知っておいた方が良いかもしれない。

In [8]:
%%sql
select
    case when sex = 'man' then TRUE else FALSE end as sex_is_man
    ,case when sex = 'woman' then TRUE else FALSE end as sex_is_woman
from
    customer_tb
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sex_is_man,sex_is_woman
True,False
True,False
False,True
True,False
True,False
True,False
True,False
False,True
False,True
False,True


#### R (Awesome)

- caretパッケージのdummyVars関数を使う。
- R言語は多くがfactor型をサポートしているらしいので、使用頻度は低いが、いざというときのために覚えて置いた方が良い。

#### Python (Awesome)
- get_dummies関数でダミー変数化が可能
- drop_firstはデフォルトFalseなので安心してください。

In [9]:
customer_df['sex_c'] = pd.Categorical(customer_df['sex'])
dummy_vars = pd.get_dummies(customer_df['sex_c'], drop_first=False)
dummy_vars.head(10)

,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
5,1,0
6,1,0
7,0,1
8,0,1
9,0,1


- Categoricalを経由する必要があるのか？という素朴な疑問があるが…文字列はそのまま行ける。

In [10]:
print(customer_df['sex'].dtype)
dummy_vars = pd.get_dummies(customer_df['sex'], drop_first=False)
dummy_vars.head(10)

object


,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
5,1,0
6,1,0
7,0,1
8,0,1
9,0,1


- 整数型もOK。

In [11]:
customer_df['generation'] = customer_df['age'] // 10 * 10
dummy_vars = pd.get_dummies(customer_df['generation'], drop_first=False)
dummy_vars

,20,30,40,50,60,70,80
0,0,0,1,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
995,0,0,1,0,0,0,0
996,0,1,0,0,0,0,0
997,0,1,0,0,0,0,0
998,0,0,1,0,0,0,0


- 複数カラム指定は…？（あれれ？）

In [12]:
# customer_df['generation'] = pd.Categorical(customer_df['age'] // 10 * 10)
dummy_vars = pd.get_dummies(customer_df[['sex','generation']], drop_first=False)
dummy_vars

,generation,sex_man,sex_woman
0,40,1,0
1,30,1,0
2,40,0,1
3,40,1,0
4,30,1,0
...,...,...,...
995,40,1,0
996,30,1,0
997,30,0,1
998,40,0,1


- category型にしたら複数カラムも良い感じ！？謎である。

In [13]:
customer_df['generation'] = pd.Categorical(customer_df['age'] // 10 * 10)
dummy_vars = pd.get_dummies(customer_df[['sex','generation']], drop_first=False)
dummy_vars

,sex_man,sex_woman,generation_20,generation_30,generation_40,generation_50,generation_60,generation_70,generation_80
0,1,0,0,0,1,0,0,0,0
1,1,0,0,1,0,0,0,0,0
2,0,1,0,0,1,0,0,0,0
3,1,0,0,0,1,0,0,0,0
4,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
995,1,0,0,0,1,0,0,0,0
996,1,0,0,1,0,0,0,0,0
997,0,1,0,1,0,0,0,0,0
998,0,1,0,0,1,0,0,0,0


#### 補足：Python + Category Encoders (おーさむ？)

- 個人的には、Category Encodersがおすすめ
- get_dummiesで学習時と推論時にカテゴリが違う場合とかどうするん？とか悩みが深そうな部分は、これが解決してくれる。
- 詳しくは書籍から離れてしまうので、以下を見てみてちょ。
  - [Category Encodersでカテゴリ特徴量をストレスなく変換する - Qiita](https://qiita.com/Hyperion13fleet/items/afa49a84bd5db65ffc31)
- 一番良いのは、後述するエンコード方式と同じAPIで(scikit-learn風APIで)実装できる点

In [14]:
# !pip install category_encoders
import category_encoders as ce

ohe = ce.OneHotEncoder()
ohe.fit_transform(customer_df[["sex","generation"]])

,sex_1,sex_2,generation_1,generation_2,generation_3,generation_4,generation_5,generation_6,generation_7
0,1,0,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
995,1,0,1,0,0,0,0,0,0
996,1,0,0,1,0,0,0,0,0
997,0,1,0,1,0,0,0,0,0
998,0,1,1,0,0,0,0,0,0


### 009_category/003:カテゴリ値の集約

- 頻度の低いカテゴリ型がある
  - これを学習データとすると、少ないデータから特性を学習するため、過学習に陥りやすくなる。
- カテゴリの種類数が多すぎる
  - 分析などがしづらい。
- まとめる場合のまとめ方は注意深く検討が必要。
  - 特性の近いものをまとめた方が良い。

<span style="background-color:yellow;">問３：10歳区切りでカテゴリ化し、60歳以上は一つに集約せよ</span>

#### SQL (Awesome)

- case文使っちゃいます。
- 教科書はORでつないでいるけど、INなどでも良さそう。

In [15]:
%%sql
with customer_tb_with_age_rank as (
    select
        *
        , cast( floor( age / 10 ) * 10 as text) as age_rank
    from
        customer_tb
)
select
    customer_id, age, sex, home_latitude, home_longitude
    , case when age_rank in ('60', '70', '80') then '60才以上' else age_rank end as age_rank
from
    customer_tb_with_age_rank
limit
    20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


customer_id,age,sex,home_latitude,home_longitude,age_rank
c_1,41,man,35.092193,136.512347,40
c_2,38,man,35.325076,139.410551,30
c_3,49,woman,35.120543,136.511179,40
c_4,43,man,43.034868,141.240314,40
c_5,31,man,35.102661,136.523797,30
c_6,52,man,34.440768,135.390487,50
c_7,50,man,43.015758,141.231321,50
c_8,65,woman,38.201268,140.465961,60才以上
c_9,36,woman,33.3228,130.330689,30
c_10,34,woman,34.290414,132.302601,30


- 90歳が来た場合どうすんねんという話は、書籍でもフォローされている。
  - `>`は`>=`に要修正

In [16]:
%%sql
select
    *,
    case when age >= 60 then
        '60才以上'
    else
        cast( floor( age / 10 ) * 10 as text)
    end as age_rank
from
    customer_tb
limit
    20

 * postgresql://padawan:***@db:5432/dsdojo_db
20 rows affected.


customer_id,age,sex,home_latitude,home_longitude,age_rank
c_1,41,man,35.092193,136.512347,40
c_2,38,man,35.325076,139.410551,30
c_3,49,woman,35.120543,136.511179,40
c_4,43,man,43.034868,141.240314,40
c_5,31,man,35.102661,136.523797,30
c_6,52,man,34.440768,135.390487,50
c_7,50,man,43.015758,141.231321,50
c_8,65,woman,38.201268,140.465961,60才以上
c_9,36,woman,33.3228,130.330689,30
c_10,34,woman,34.290414,132.302601,30


#### R (Awesome)

- マスターデータを書き換えたりする必要があるようだ

#### Python (Awesome)
- Rと同様、マスターデータを書き換えたりする必要がある。
  - `.cat.add_categories`と`.cat.remove_unused_categories`を使う。
- floorは使わず`//`でやった。
- inplaceは`Deprecated since version 1.3.0.`らしい。
  - https://pandas.pydata.org/docs/reference/api/pandas.Series.cat.add_categories.html

In [17]:
customer_df['age_rank'] = pd.Categorical(customer_df['age'] // 10 * 10)
print(customer_df['age_rank'].cat.categories)
# # customer_df['age_rank'].cat.add_categories(['60才以上'], inplace=True) # inplaceはdfの更新をする場合に指定するがwarningになるよ。
customer_df['age_rank'] = customer_df['age_rank'].cat.add_categories(['60才以上'])
print(customer_df['age_rank'].cat.categories)
# # customer_df.loc[ customer_df['age_rank']

Int64Index([20, 30, 40, 50, 60, 70, 80], dtype='int64')
Index([20, 30, 40, 50, 60, 70, 80, '60才以上'], dtype='object')


- どちらが分かりやすいというわけでもないが、queryではこう書ける。

In [18]:
customer_df.loc[ customer_df.query('age_rank in (60, 70, 80)').index, 'age_rank'] = '60才以上'
customer_df.head(10)

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,generation,age_rank
0,c_1,41,man,35.092193,136.512347,True,man,40,40
1,c_2,38,man,35.325076,139.410551,True,man,30,30
2,c_3,49,woman,35.120543,136.511179,False,woman,40,40
3,c_4,43,man,43.034868,141.240314,True,man,40,40
4,c_5,31,man,35.102661,136.523797,True,man,30,30
5,c_6,52,man,34.440768,135.390487,True,man,50,50
6,c_7,50,man,43.015758,141.231321,True,man,50,50
7,c_8,65,woman,38.201268,140.465961,False,woman,60,60才以上
8,c_9,36,woman,33.322800,130.330689,False,woman,30,30
9,c_10,34,woman,34.290414,132.302601,False,woman,30,30


In [19]:
customer_df['age_rank'] = customer_df['age_rank'].cat.remove_unused_categories()
print(customer_df['age_rank'].cat.categories)
customer_df.head(10)

Index([20, 30, 40, 50, '60才以上'], dtype='object')


,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,generation,age_rank
0,c_1,41,man,35.092193,136.512347,True,man,40,40
1,c_2,38,man,35.325076,139.410551,True,man,30,30
2,c_3,49,woman,35.120543,136.511179,False,woman,40,40
3,c_4,43,man,43.034868,141.240314,True,man,40,40
4,c_5,31,man,35.102661,136.523797,True,man,30,30
5,c_6,52,man,34.440768,135.390487,True,man,50,50
6,c_7,50,man,43.015758,141.231321,True,man,50,50
7,c_8,65,woman,38.201268,140.465961,False,woman,60,60才以上
8,c_9,36,woman,33.322800,130.330689,False,woman,30,30
9,c_10,34,woman,34.290414,132.302601,False,woman,30,30


### 009_category/004:カテゴリ値の組み合わせ

- 性別と年代を組み合わせ、新たなカテゴリを構成する。
- これにより、男性20代と女性20代で大きく傾向が異なっていたとしても、その変化を表現できるようになる。
- デメリットとしては、
  - ダミー変数化した場合に、データ量が増える。
    - 結合前: 2 + 7世代 = 9個分のカラムのデータ量
    - 結合後: 2 x 7世代 = 14個分のカラムのデータ量
  - 結合により、サンプルの少ないカテゴリが出現する可能性がある。
    - サンプルが少ない場合、過学習の可能性がある。

<span style="background-color:yellow;">問４：性別と年齢を結合したカテゴリを作成せよ</span>

#### SQL (Awesome)
- 文字列操作で結合する
- `||`で結合できるっぽい（知らなかった）

In [20]:
%%sql
select
    *
    , sex || '_' || cast(floor(age / 10) * 10 as text) as sex_and_age
from
    customer_tb
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,age,sex,home_latitude,home_longitude,sex_and_age
c_1,41,man,35.092193,136.512347,man_40
c_2,38,man,35.325076,139.410551,man_30
c_3,49,woman,35.120543,136.511179,woman_40
c_4,43,man,43.034868,141.240314,man_40
c_5,31,man,35.102661,136.523797,man_30
c_6,52,man,34.440768,135.390487,man_50
c_7,50,man,43.015758,141.231321,man_50
c_8,65,woman,38.201268,140.465961,woman_60
c_9,36,woman,33.3228,130.330689,woman_30
c_10,34,woman,34.290414,132.302601,woman_30


#### R (Awesome)

- pasteで結合、mutateは列を追加する関数っぽい。

#### Python (Awesome)

- applyはちょっと遅いし、axis=1としないと列に対して使えないのが難点。

In [21]:
%%timeit
customer_df[['sex','age']].apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10), axis=1)

8.67 ms ± 174 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
customer_df['sex_and_age'] = pd.Categorical(
    customer_df[['sex','age']].apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1] / 10) * 10), axis=1)
)
customer_df.head(10)

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,generation,age_rank,sex_and_age
0,c_1,41,man,35.092193,136.512347,True,man,40,40,man_40.0
1,c_2,38,man,35.325076,139.410551,True,man,30,30,man_30.0
2,c_3,49,woman,35.120543,136.511179,False,woman,40,40,woman_40.0
3,c_4,43,man,43.034868,141.240314,True,man,40,40,man_40.0
4,c_5,31,man,35.102661,136.523797,True,man,30,30,man_30.0
5,c_6,52,man,34.440768,135.390487,True,man,50,50,man_50.0
6,c_7,50,man,43.015758,141.231321,True,man,50,50,man_50.0
7,c_8,65,woman,38.201268,140.465961,False,woman,60,60才以上,woman_60.0
8,c_9,36,woman,33.322800,130.330689,False,woman,30,30,woman_30.0
9,c_10,34,woman,34.290414,132.302601,False,woman,30,30,woman_30.0


- floorじゃなくて`//`でやった方が速そう

In [23]:
%%timeit
customer_df[['sex','age']].apply(lambda x: f'{x[0]}_{x[1] // 10 * 10}', axis=1)

7.27 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
customer_df['sex_and_age'] = pd.Categorical(
    customer_df[['sex','age']].apply(lambda x: f'{x[0]}_{x[1] // 10 * 10}', axis=1)
)
customer_df.head(10)

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,generation,age_rank,sex_and_age
0,c_1,41,man,35.092193,136.512347,True,man,40,40,man_40
1,c_2,38,man,35.325076,139.410551,True,man,30,30,man_30
2,c_3,49,woman,35.120543,136.511179,False,woman,40,40,woman_40
3,c_4,43,man,43.034868,141.240314,True,man,40,40,man_40
4,c_5,31,man,35.102661,136.523797,True,man,30,30,man_30
5,c_6,52,man,34.440768,135.390487,True,man,50,50,man_50
6,c_7,50,man,43.015758,141.231321,True,man,50,50,man_50
7,c_8,65,woman,38.201268,140.465961,False,woman,60,60才以上,woman_60
8,c_9,36,woman,33.322800,130.330689,False,woman,30,30,woman_30
9,c_10,34,woman,34.290414,132.302601,False,woman,30,30,woman_30


- 普通にzipしてリスト内包表記する方が早い。

In [25]:
%%timeit
[f'{x0}_{x1//10*10}' for x0, x1 in zip(customer_df['sex'], customer_df['age'])]

314 µs ± 24.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
customer_df['sex_and_age'] = pd.Categorical(
    [f'{x0}_{x1//10*10}' for x0, x1 in zip(customer_df['sex'], customer_df['age'])]
)
customer_df.head(10)

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,generation,age_rank,sex_and_age
0,c_1,41,man,35.092193,136.512347,True,man,40,40,man_40
1,c_2,38,man,35.325076,139.410551,True,man,30,30,man_30
2,c_3,49,woman,35.120543,136.511179,False,woman,40,40,woman_40
3,c_4,43,man,43.034868,141.240314,True,man,40,40,man_40
4,c_5,31,man,35.102661,136.523797,True,man,30,30,man_30
5,c_6,52,man,34.440768,135.390487,True,man,50,50,man_50
6,c_7,50,man,43.015758,141.231321,True,man,50,50,man_50
7,c_8,65,woman,38.201268,140.465961,False,woman,60,60才以上,woman_60
8,c_9,36,woman,33.322800,130.330689,False,woman,30,30,woman_30
9,c_10,34,woman,34.290414,132.302601,False,woman,30,30,woman_30


### 009_cateogory/005:カテゴリ型の数値化

- 集約のさらなる例として数値化があるが、基本的にはデータ本来の意味を失うため、非推奨と記述されているそう。
- 例：製造レコードの製造物の品種を数値化
  - 品種ごとに出現回数をカウントしてカテゴリ値の代わりに使用。（Category EncodersではCountEncoding）
  - 品種ごとの障害発生率を計算しカテゴリ値の代わりに使用。（おお？なるほど）
  - 前述の障害発生率を基準にランクを計算して使用。（Category Encodersでは、一種のOrdinalEncodingかな）
- リークを起こさずに実施することが難しいので慎重に検証すること。
  - 使うことは結構ある気がするので、危ないので使わないで、というよりは検証が重要かなと思っている。
  - 目的変数を使ったエンコーディングでTargetEncodingとか黒魔術的な方法もある。
    - [Target Encodingとは？3種類のターゲットエンコーディングとPython実装方法を徹底解説](https://www.codexa.net/target_encoding/)

<span style="background-color:yellow;">問４：製造レコードについて、type別の平均障害率に変換せよ</span>

#### SQL (Awesome)

- fault_flgを目的変数とした場合の、leave-one-outなTargetEncodingのようです。
- 注釈[2]で説明ありましたね。

<p style="background-color:#EEEEEE;">
    製品種別ごとの平均障害率を利用して障害予測モデルを学習する場合、<br>
    自身のレコードを除く方がモデルの精度を上げられることが多いです。<br>
    なぜなら、全レコードの製品種別ごとの平均障害率には予測すべき値の情報が含まれてしまっているからです。<br>
    その結果、弱いリークによる過学習を引き起こしてしまいます。<br>
    ただし、自身のレコードの影響がほとんどない程度に平均障害率を算出する元のレコード数が多ければ影響ありません。<br>
    このように、副作用なく予測する値を利用した説明変数を予測モデルに利用することは非常に難しいです。<br>
    危険性を理解して、十分に対処できる方以外にはお勧めしません。<br>
    また、機械学習モデルを学習したあとの運用時には製品種別ごとの平均障害率を利用した障害予測モデルを活用して予測を行うときには、<br>
    製品種別ごとの平均障害率は学習データ全体の製品種別ごとの平均障害率を利用しましょう。<br>
</p>

In [27]:
%%sql
with type_mst as (
    select
        type
        , count(*) as record_cnt
        , sum(case when fault_flg then 1 else 0 end) as fault_cnt
    from
        production_tb
    group by type
)
select
    base.*
    , cast(t_mst.fault_cnt - (case when fault_flg then 1 else 0 end) as float) / (t_mst.record_cnt - 1) as type_fault_rate
from
    production_tb base
inner join type_mst t_mst
    on base.type = t_mst.type
limit
    10

 * postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


type,length,thickness,fault_flg,type_fault_rate
E,274.0273827080609,40.24113135955541,False,0.061224489795918366
D,86.31926860506081,16.906714630016268,False,0.03271028037383177
E,123.94038830419984,1.0184619943950775,False,0.061224489795918366
B,175.5548859820581,16.41492419553766,False,0.034482758620689655
B,244.93473950709702,29.061080805480326,False,0.034482758620689655
B,226.42651402498535,39.76380137107543,False,0.034482758620689655
C,331.63767138081164,16.83556687876311,False,0.0761904761904762
A,200.86544412142973,12.184256052452268,False,0.05472636815920398
C,276.38663112594,29.899611485794143,False,0.0761904761904762
E,168.44121347494632,1.2659187080578818,False,0.061224489795918366


#### R (Awesome)
- Rさんは結構シンプルに書けてますね。一番シンプルかも。

#### Python (Awesome)
- Pythonは結構地道なので、やっぱりCategory Encodersをオススメする。

In [28]:
fault_cnt_per_type = production_df.query('fault_flg').groupby('type')['fault_flg'].count()
type_cnt = production_df.groupby('type')['fault_flg'].count()
production_df['type_fault_rate'] = production_df[['type', 'fault_flg']] \
    .apply(lambda x:
           (fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] - 1)
           , axis=1)
production_df

,type,length,thickness,fault_flg,type_fault_rate
0,E,274.027383,40.241131,False,0.061224
1,D,86.319269,16.906715,False,0.032710
2,E,123.940388,1.018462,False,0.061224
3,B,175.554886,16.414924,False,0.034483
4,B,244.934740,29.061081,False,0.034483
...,...,...,...,...,...
995,C,363.214163,48.369483,False,0.076190
996,D,134.773797,26.861665,False,0.032710
997,B,231.174985,7.087471,False,0.034483
998,D,81.613510,5.716271,False,0.032710


#### 補足：Python + Category Encoders (おーさむ？)

- これだけでOK。

In [29]:
# !pip install category_encoders
import category_encoders as ce

loo_enc = ce.LeaveOneOutEncoder()
production_df["type_fault_rate"] = loo_enc.fit_transform(production_df["type"], production_df["fault_flg"])
production_df

,type,length,thickness,fault_flg,type_fault_rate
0,E,274.027383,40.241131,False,0.061224
1,D,86.319269,16.906715,False,0.032710
2,E,123.940388,1.018462,False,0.061224
3,B,175.554886,16.414924,False,0.034483
4,B,244.934740,29.061081,False,0.034483
...,...,...,...,...,...
995,C,363.214163,48.369483,False,0.076190
996,D,134.773797,26.861665,False,0.032710
997,B,231.174985,7.087471,False,0.034483
998,D,81.613510,5.716271,False,0.032710


### 009_category/006:カテゴリ型の補完

- 欠損値の対応である。
- その種類
  - 固定値で補完（非推奨）
  - 集計値（最頻値など）で補完（欠損が多い場合は非推奨）
  - 欠損してないカラムから欠損しているカラムを予測して補完する。多重代入法が使われることもある。
  - 時系列で補完
  - 多重代入法
  - 最尤法

<span style="background-color:yellow;">問５：欠損した製造レコードについて、KNNを使用して補完せよ</span>

- kNNとは？
  - [kNNの説明とその実装 - Qiita](https://qiita.com/oirom/items/22ccb7c0139dce925f43)
  - 要するに、ある空間で、対象の点に最も近いk個のサンプルの多数決で、対象の点の値を決める。
  - なので、この空間を構成する場合は、事前に正規化してあげた方が良い。
- `対象のデータセットは、fault_flgに欠損が存在する製造レコードです。`と書いてあるが、typeが欠損するレコードのこと。

#### R (Awesome)

- classパッケージのknn関数で実現

#### Python (Awesome)

- sklearnで実行する。
- index.differenceは初見だったが、引数値違うindexのみを抽出できるみたい。

In [30]:
from sklearn.neighbors import KNeighborsClassifier

# データ内は'None'文字列なので、np.nanに置き換える
production_missin_category_df.replace('None', np.nan, inplace=True)

# train用に欠損していない部分のみ抽出
train = production_missin_category_df.dropna(subset=['type'])

# testを抽出
test = production_missin_category_df[production_missin_category_df['type'].isnull()] # これでも可能
# test = production_missin_category_df.loc[production_missin_category_df.index.difference(train.index), :] # 書籍はこちら

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train[['length','thickness']], train['type'])

test = test.reset_index(drop=True) # warningが出るので。たぶんpredictの戻り値がnumpy.arrayだから？
test['type'] = kn.predict(test[['length','thickness']])
test

,type,length,thickness,fault_flg
0,E,276.386631,29.899611,False
1,E,263.844324,34.664251,False
2,E,129.364736,21.346752,False
3,A,203.378972,30.286454,False
4,E,157.463166,11.166165,False
...,...,...,...,...
95,A,130.088061,0.207250,False
96,E,284.562824,49.211790,False
97,B,264.130761,4.560416,False
98,A,182.252364,33.314305,False
